In [3]:
conda install -c anaconda nltk

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [9]:

import sklearn 
sklearn.__version__

'0.23.2'

In [13]:
# import dependencies
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/skirwai/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
# Data preprocessing

In [18]:
news_dataset = pd.read_csv('fake-news/train.csv')

In [19]:
news_dataset.shape

(20800, 5)

In [20]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [21]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
# cos we have large dataset to no need to replace 
# null values with anyother values

In [24]:
# replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [25]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [26]:
# merging author name and title
news_dataset['context'] = news_dataset['author'] + ' ' + news_dataset['title']

In [27]:
news_dataset['context']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: context, Length: 20800, dtype: object

In [28]:
# seperating data & label
X = news_dataset.drop(['label'], axis=1)
Y = news_dataset['label']

In [29]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming - Process of reducing word to it root word means in 
example : 
actor, actress, acting ---> act


In [30]:
post_stem = PorterStemmer()

In [35]:
import re
def stem(word):
    stem_word = re.sub('[^a-zA-Z]',' ', word)
    stem_word = stem_word.lower()
    stem_word = stem_word.split()
    stem_word = [post_stem.stem(word) for word in stem_word 
                 if not word in stopwords.words('english')]
    stem_word = ' '.join(stem_word)
    return stem_word
    

In [36]:
news_dataset['context'] = news_dataset['context'].apply(stem)

In [37]:
print(news_dataset['context'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: context, Length: 20800, dtype: object


In [39]:
# seperating data and label
X = news_dataset['context'].values
Y = news_dataset['label'].values

In [40]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [41]:
print(X.shape,Y.shape)

(20800,) (20800,)


In [42]:
# converting textual data to numeriacl data
vertorizer = TfidfVectorizer()

vertorizer.fit(X)

X = vertorizer.transform(X)

In [43]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [44]:
# splitting dataset to train & test

In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2,
                                                    stratify = Y,
                                                    random_state=2)

In [48]:
print(X.shape,X_train.shape, X_test.shape)

(20800, 17128) (16640, 17128) (4160, 17128)


In [78]:
print(X_train)

  (0, 16996)	0.09995656818816077
  (0, 16928)	0.3360072514082535
  (0, 15295)	0.09807665903342763
  (0, 13914)	0.3334797245354899
  (0, 13014)	0.2680313811122545
  (0, 12501)	0.3929876463935473
  (0, 11936)	0.24142639024498436
  (0, 10306)	0.09662001419895176
  (0, 10219)	0.3019527708144002
  (0, 3155)	0.3400831511004003
  (0, 2794)	0.3776836172783757
  (0, 336)	0.3360072514082535
  (1, 16996)	0.07263181421455335
  (1, 15424)	0.22579404836928033
  (1, 15417)	0.26613170238131584
  (1, 15295)	0.07126580880898774
  (1, 13453)	0.3387500815971264
  (1, 11421)	0.3084666283145136
  (1, 10306)	0.07020736153621741
  (1, 10061)	0.24924889065491332
  (1, 9856)	0.3387500815971264
  (1, 5313)	0.2768869285533855
  (1, 2761)	0.3387500815971264
  (1, 2207)	0.3387500815971264
  (1, 956)	0.2855580802628186
  :	:
  (16637, 10008)	0.32617532466383187
  (16637, 9588)	0.184164927958451
  (16637, 8153)	0.3214731053599403
  (16637, 5009)	0.3538476463585765
  (16637, 1868)	0.40919228974806576
  (16637, 364)	0.

In [49]:
print(Y.shape,Y_train.shape, Y_test.shape)

(20800,) (16640,) (4160,)


trainning model

In [50]:
model = LogisticRegression()

In [51]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy_score onn trainning data

In [52]:
X_train_predict = model.predict(X_train)
training_accuracy = accuracy_score(X_train_predict, Y_train)
print(training_accuracy)

0.9865985576923076


In [53]:
X_test_predict = model.predict(X_test)
test_accuracy = accuracy_score(X_test_predict, Y_test)
print(test_accuracy)

0.9790865384615385


In [54]:
# build predictive system

In [74]:
X_news = X_test[5]

prediction = model.predict(X_news)
print(prediction)

if prediction[0] == 0:
    print('news is real')
else:
    print('news is fake')

[1]
news is fake


In [75]:
print(Y_test[5])

1


In [63]:
list(X_test_predict)

[1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,


In [65]:
test_dataset = pd.read_csv('fake-news/test.csv')

In [66]:
test_dataset = test_dataset.fillna('')

In [68]:
# merging author name and title
test_dataset['context'] = test_dataset['author'] + ' ' + test_dataset['title']

In [69]:
test_dataset['context'] = test_dataset['context'].apply(stem)

In [70]:
X = test_dataset['context'].values

In [71]:
# converting textual data to numeriacl data
vertorizer = TfidfVectorizer()

vertorizer.fit(X)

X = vertorizer.transform(X)

In [76]:
print(X)

  (0, 9086)	0.08756899768504829
  (0, 8618)	0.2951833364547817
  (0, 8391)	0.11042633230471807
  (0, 8258)	0.35551158241433845
  (0, 8217)	0.08565162006825207
  (0, 7825)	0.35551158241433845
  (0, 7815)	0.3392484573767603
  (0, 7632)	0.35551158241433845
  (0, 7416)	0.29990759064553757
  (0, 6435)	0.35551158241433845
  (0, 5528)	0.0847731814904578
  (0, 4746)	0.35551158241433845
  (0, 1960)	0.20372848702181867
  (1, 8826)	0.4600630644689105
  (1, 8135)	0.35260162515090443
  (1, 7823)	0.36689240142839846
  (1, 6999)	0.30028557064663775
  (1, 6583)	0.40045820302192875
  (1, 5485)	0.37981022352550325
  (1, 186)	0.36689240142839846
  (2, 8964)	0.295064720779198
  (2, 8765)	0.28434502467517675
  (2, 7741)	0.2982257606996168
  (2, 6196)	0.2207192031076798
  (2, 5596)	0.3312058036253688
  :	:
  (5196, 795)	0.21762533251043967
  (5196, 24)	0.2798172726761391
  (5197, 9086)	0.12439214769277508
  (5197, 8237)	0.3546424451054057
  (5197, 8217)	0.1216685043258708
  (5197, 7942)	0.46551291399479516


In [73]:
new_X_test_predict = model.predict(X)
print(new_X_test_predict)

ValueError: X has 9142 features per sample; expecting 17128

In [ ]:
predictions_df = pd.DataFrame(
    list(zip(list(X_test_predict), lst2)),
               columns =['Name', 'val'],
    columns=unigrams_pipeline.classes_